In this notebook we will divide the general task of building the RAG system into smaller parts and test solutions for every one of them.

As a data source we picked full text of the book "THE COUNT OF MONTE CRISTO" english version.

# Chunking

As the base of the RAG system is a typical LLM, it has limited contex window, so we can't give whole text at once, hence, we need to divide it into chunks.

In [198]:
import os
import pandas as pd
from langchain_text_splitters import CharacterTextSplitter
import seaborn as sns
import ast

In [109]:
#from google.colab import drive
#drive.mount('/content/drive')

In [110]:
DATA_FILE_PATH = os.path.join(os.path.dirname(os.getcwd()), "data\\CountMonteCristoFull.txt")

In [111]:
with open(DATA_FILE_PATH, "r", encoding="utf-8") as f:
    data_corpus = f.read()

The whole text consists of 2646643 characters

In [112]:
len(data_corpus)

2646643

Based on the information, provided on HugginFace [website](https://huggingface.co/meta-llama/Meta-Llama-3-8B), llama3-8b-8192 models(which was chosen as LLM for this system) context window is the size 8192. This means that we should divide the whole dataset into parts about the size of 6000-6500 tokens.\
Besides that, to ensure integrity of models understanding of the text, these parts will be overlapping by 400-600 tokens to ensure that connections between consecutive chunks are retained.\
As we got from experimenting, these are the correlations between the lenghts of text chunks and tokens chunks
* 5500 len text chunk + 600 len overlap ~ 1200-1450 len token chunks(this variant was tested, but the system responces were the worst among all tests)
* 8000 len text chunk + 800 len overlap ~ 1700-2100 len token chunks(results provided by the system were better than with 5.5k text characters split)
* 10000 len text chunk + 1000 len overlap ~ 2300-2600 len token chunks(this variant was tested and the system responces were almost always consistently good)

After further analyzing chapters size, which are between 10k-60k symbols, the decision was made to split text into chunks of the size 10k symbols(2.3k-2.6 tokens).

In [113]:
splitter = CharacterTextSplitter(separator="\n\n", chunk_size=10_000, chunk_overlap=1_000)
text_chunks = splitter.create_documents([data_corpus])

In [114]:
print('Example of text chunk:', text_chunks[0].page_content,
      '\nNumber of chunks:', len(text_chunks))

Example of text chunk: ﻿The Project Gutenberg eBook of The Count of Monte Cristo
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: The Count of Monte Cristo

Author: Alexandre Dumas
        Auguste Maquet

Release date: January 1, 1998 [eBook #1184]
                Most recently updated: February 4, 2024

Language: English

Credits: Anonymous Project Gutenberg Volunteers, Dan Muller and David Widger


*** START OF THE PROJECT GUTENBERG EBOOK THE COUNT OF MONTE CRISTO ***
THE COUNT OF MONTE CRISTO

by Alexandre Dumas [père]

0009m 

0011m 

0019m 

Contents


 VOLUME ONE
Chapter 1. Marsei

In [115]:
DENSE_RETRIEVER_MODEL_NAME = "all-MiniLM-L6-v2"
CROSS_ENCODER_MODEL_NAME = 'cross-encoder/ms-marco-MiniLM-L-12-v2'
LLM_CORE_MODEL_NAME = "groq/llama3-8b-8192"

For further pusposes, when the citation part comes up, we need not only to show the user the part of the text that may answer his question, but also provide him the approximate location of the answer. For this, we will attach metadata for each text chunk, that will tell which Chapter in the book this text belongs to. Chapter sizes are between 10k-60k characters in length, so there are no chances that several chapter names would occur in one chunk.

In [116]:
prev_chapter_name = ''
for chunk in text_chunks:
    chunk.metadata['belongs_to'] = set()
    curr_chapter_name = ''
    index_start_chapter_name = chunk.page_content.find('Chapter')

    if index_start_chapter_name == -1:
        curr_chapter_name = prev_chapter_name
    else:
        # if prev_chapter_name is not empty and next chapter start further than first 40% of the chunk.
        # This means that the name of the prev chapter isn't in this chunk, but relevant info can be found.
        if prev_chapter_name != '' and index_start_chapter_name > int(len(chunk.page_content) * 0.4):
            chunk.metadata['belongs_to'].add(prev_chapter_name)

        index_end_chapter_name = chunk.page_content.find('\n', index_start_chapter_name)
        curr_chapter_name = chunk.page_content[index_start_chapter_name:index_end_chapter_name]
        prev_chapter_name = curr_chapter_name
    chunk.metadata['belongs_to'].add(curr_chapter_name)

    chunk.metadata['belongs_to'] = list(chunk.metadata['belongs_to'])

In [117]:
text_chunks[:5]

[Document(page_content='\ufeffThe Project Gutenberg eBook of The Count of Monte Cristo\n    \nThis ebook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever. You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this ebook or online\nat www.gutenberg.org. If you are not located in the United States,\nyou will have to check the laws of the country where you are located\nbefore using this eBook.\n\nTitle: The Count of Monte Cristo\n\nAuthor: Alexandre Dumas\n        Auguste Maquet\n\nRelease date: January 1, 1998 [eBook #1184]\n                Most recently updated: February 4, 2024\n\nLanguage: English\n\nCredits: Anonymous Project Gutenberg Volunteers, Dan Muller and David Widger\n\n\n*** START OF THE PROJECT GUTENBERG EBOOK THE COUNT OF MONTE CRISTO ***\nTHE COUNT OF MONTE CRISTO\n\nby Alexandre Dumas [père]\n\n0009m \n\n0011m \n\n0019m \n\n

Here we will define an additional function to perform text cleaning. Text will be cleaned by removing punctuation, converting to lowercase, removing special characters and extra whitespace.

In [118]:
import re
import string

def clean_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

Now, when all the preperations of the dataset are done, we can form a new version of the documents collection.

In [119]:
chunked_data_corpus = []

for index, chunk in enumerate(text_chunks):
    chunked_data_corpus.append({
        'raw_text': chunk.page_content,
        'cleaned_text': clean_text(chunk.page_content),
        'chapter_name': chunk.metadata['belongs_to']
    })

In [120]:
chunked_data_corpus_df = pd.DataFrame(chunked_data_corpus)
chunked_data_corpus_df.head()

,raw_text,cleaned_text,chapter_name
0,﻿The Project Gutenberg eBook of The Count of M...,the project gutenberg ebook of the count of mo...,[Chapter 1. Marseilles—The Arrival]
1,"“Why, you see, Edmond,” replied the owner, who...",why you see edmond replied the owner who appea...,[Chapter 1. Marseilles—The Arrival]
2,"“Sometimes one and the same thing,” said Morre...",sometimes one and the same thing said morrel w...,"[Chapter 2. Father and Son, Chapter 1. Marseil..."
3,“Whom does this belong to?” he inquired.\n\n“T...,whom does this belong to he inquired to me to ...,"[Chapter 2. Father and Son, Chapter 3. The Cat..."
4,“Really; and you think this cousin pays her at...,really and you think this cousin pays her atte...,[Chapter 3. The Catalans]


In [121]:
chunked_data_corpus_df.to_csv('../data/chunked_data_corpus.csv', index=False)

In [122]:
# chunked_data_corpus_df = pd.read_csv('/content/drive/MyDrive/chunked_data_corpus.csv')
chunked_data_corpus_df = pd.read_csv('../data/chunked_data_corpus.csv')
chunked_data_corpus_df = chunked_data_corpus_df.to_dict('records')

In [123]:
!pip install litellm python-dotenv

In [124]:
from litellm import completion
from dotenv import load_dotenv
import os

load_dotenv()

# GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GROQ_API_KEY = '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'

# Retriever

The algorithm, that typicaly used to implement retriever functionality is simple TF-IDF score. Here, we will be using more efficient its implementation BM25.\
They are both information retrieval methods used to rank documents based on their relevance to a query.\
Key differences between them are:
* Term Saturation: The impact of term frequency diminishes as term frequency increases.
* Document Length Normalization: Adjusts scores based on document length using length normalization parameter and average document length.

Based on this, BM25 is more suitable for task like RAG systems.

Aside from TF-IDF kind of score to compare query with the documents, that is rather general, we can also use dense retriever.\
Dense retriever(which will be implemented as bi-encoder) is the one that catches semantical filling of the text, so basically can compare text in different languages with the same meanings by computing vector embeddings for each document and query and compare their similarity by using smth like cosine similarity.\
To implement dense retriever, often models like [sentence bert](https://arxiv.org/abs/1908.10084) or [LaBSE](https://arxiv.org/abs/2007.01852) are chosen.
For our system, we will try smth more simple but still effective, like "all-MiniLM-L6-v2" model.\
Using retriever we will firstly pick 20%(60 docs) of the most relevant docs out of all using BM25. Then, using dense retriever, we will pick 50%(30 docs) out of all picked by BM25 as most similar.
Besides that, for time efficiency, we will precompute chunks embeddings online and simply pick them in dense retrieval part.

# Reranker

As both retirievers(bm25 and dense) capture only key words and general information about the text respectivelly, the retrieved documents can be pretty far from query semanticaly. To overcome this issue, we can use reranker, which will be implemented as cross-encoder, that encodes pairs od document and query into numerical vectors and captures specific information that is asked in a query. This approach is less cost efficient, so it can't be performed on the whole document corpus. So, we will pass only 30 docs(10% of the whole corpus), which were returned by retriever. As a result, we will pick 2 most relevant documents to pass to the LLM context.\
Exactly 2 documents were considered maximum to pick, cause each document is approximately 1700-2100 long in tokens. LLM's context window is 8000 tokens, so picking 2 most relevant docs we will be left with 2000 tokens for query and answer.

In [125]:
!pip install rank_bm25

In [126]:
from rank_bm25 import BM25Okapi
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util, CrossEncoder

In [127]:
class HybridRetrieverReranker:
    def __init__(self, dataset, dense_model_name=DENSE_RETRIEVER_MODEL_NAME, cross_encoder_model=CROSS_ENCODER_MODEL_NAME):
        self.dataset = dataset
        self.bm25_corpus = [entry['cleaned_text'] for entry in dataset]
        self.tokenized_corpus = [chunk.split() for chunk in self.bm25_corpus]
        self.bm25 = BM25Okapi(self.tokenized_corpus)

        self.dense_model = SentenceTransformer(dense_model_name)
        self.cross_encoder = CrossEncoder(cross_encoder_model)

        self.dataset = self.__calculate_save_embeddings__(self.dataset)


    def __calculate_save_embeddings__(self, dataset):
        docs_embeddings = self.dense_model.encode([entry['raw_text'] for entry in dataset], convert_to_tensor=True)
        for doc, embedding in zip(dataset, docs_embeddings):
          doc['embedding'] = embedding
        return dataset


    def bm25_retrieve(self, query, top_k=70):
        """
        Retrieve top K documents using BM25.

        Args:
            query (str): Query text.
            top_k (int): Number of top BM25 documents to retrieve.

        Returns:
            list of dict: Top K BM25 results.
        """
        cleaned_query = clean_text(query)
        query_tokens = cleaned_query.split()
        bm25_scores = self.bm25.get_scores(query_tokens)
        top_k_indices = np.argsort(bm25_scores)[::-1][:top_k]
        return [self.dataset[idx] for idx in top_k_indices]


    def dense_retrieve(self, query, candidates=None, top_n=35):
        """
        Retrieve top N documents using dense retrieval with LaBSE.

        Args:
            query (str): Query text.
            candidates (list of dict): Candidate documents from BM25.
            top_n (int): Number of top dense results to retrieve.

        Returns:
            list of dict: Top N dense results.
        """
        if candidates is None:
            candidates = self.dataset
        query_embedding = self.dense_model.encode(query, convert_to_tensor=True)
        candidate_embeddings = torch.stack([doc['embedding'] for doc in candidates])

        similarities = util.pytorch_cos_sim(query_embedding, candidate_embeddings).squeeze(0)

        top_n_indices = torch.topk(similarities, top_n).indices
        return [candidates[idx] for idx in top_n_indices]


    def rerank(self, query, candidates=None, top_n=3):
        """
        Rerank top documents using a CrossEncoder.

        Args:
            query (str): Query text.
            candidates (list of dict): Candidate documents from dense retriever.
            top_n (int): Number of top reranked results to return.

        Returns:
            list of dict: Top N reranked documents.
        """
        if candidates is None:
            candidates = self.dataset
        query_document_pairs = [(query, doc['raw_text']) for doc in candidates]
        scores = self.cross_encoder.predict(query_document_pairs)
        top_n_indices = np.argsort(scores)[::-1][:top_n]
        return [candidates[idx] for idx in top_n_indices]


    def hybrid_retrieve(self, query, enable_bm25=True, enable_dense=True, enable_rerank=True, top_k_bm25=60, top_n_dense=30, top_n_rerank=2):
        """
        Perform hybrid retrieval: BM25 followed by dense retrieval and optional reranking.

        Args:
            query (str): Query text.
            top_k_bm25 (int): Number of top BM25 documents to retrieve.
            top_n_dense (int): Number of top dense results to retrieve.
            enable_dense (bool): Whether dense retrieval should be enabled.
            enable_rerank (bool): Whether reranking should be enabled.
            top_n_rerank (int): Number of top reranked documents to return.

        Returns:
            list of dict: Final top results after hybrid retrieval and reranking.
        """
        if enable_bm25:
            bm25_results = self.bm25_retrieve(query, top_k=top_k_bm25)
        else:
            bm25_results = None

        if enable_dense:
            dense_results = self.dense_retrieve(query, bm25_results, top_n=top_n_dense)
        else:
            dense_results = bm25_results

        if enable_rerank:
            final_results = self.rerank(query, dense_results, top_n=top_n_rerank)
        else:
            final_results = dense_results

        return final_results

In [128]:
retriever = HybridRetrieverReranker(chunked_data_corpus_df)
query = "Who is Monte Cristo?"
results = retriever.hybrid_retrieve(query)

for doc in results:
  print(doc['chapter_name'], '\t', doc['raw_text'][:doc['raw_text'].index('.')], '\n')

['Chapter 46. Unlimited Credit'] 	 Chapter 46 

['Chapter 47. The Dappled Grays'] 	 Monte Cristo bowed, in sign that he accepted the proffered honor;
Danglars rang and was answered by a servant in a showy livery 



# LiteLLM library (llama3-8b-8192 model).

In [ ]:
class QuestionAnsweringBot:

    def __init__(self, docs, enable_bm25=True, enable_dense=True, enable_rerank=True, top_k_bm25=60, top_n_dense=30, top_n_rerank=2) -> None:
        self.retriever = HybridRetrieverReranker(docs)
        self.enable_bm25 = enable_bm25
        self.enable_dense = enable_dense
        self.enable_rerank = enable_rerank
        self.top_k_bm25=top_k_bm25
        self.top_n_dense=top_n_dense
        self.top_n_rerank=top_n_rerank

    def __get_answer__(self, question: str) -> str:
        PROMPT = """\
            You are an intelligent assistant designed to provide accurate and relevant answers based on the provided context.

            Rules:
            - Always analyze the provided context thoroughly before answering.
            - Respond with factual and concise information.
            - If context is ambiguous or insufficient or you can't find answer, say 'I don't know.'
            - Do not speculate or fabricate information beyond the provided context.
            - Follow user instructions on the response style(default style is detailed response if user didn't provide any specifications):
              - If the user asks for a detailed response, provide comprehensive explanations.
              - If the user requests brevity, give concise and to-the-point answers.
            - When applicable, summarize and synthesize information from the context to answer effectively.
            - Avoid using information outside the given context.
          """
        context = self.retriever.hybrid_retrieve(question, 
                                                 enable_bm25=self.enable_bm25, 
                                                 enable_dense=self.enable_dense, 
                                                 enable_rerank=self.enable_rerank,
                                                 top_k_bm25=self.top_k_bm25,
                                                 top_n_dense=self.top_n_dense,
                                                 top_n_rerank=self.top_n_rerank
                                                 )

        context_text = [doc['raw_text'] for doc in context]

        response = completion(
                                model=LLM_CORE_MODEL_NAME,
                                messages=[
                                    {"role": "system", "content": PROMPT},
                                    {"role": "user", "content": f"Context: {context_text}\nQuestion: {question}"}
                            ],
                            api_key=GROQ_API_KEY
                            )
        return response, context

    def form_response(self, question):
      llm_response, context = self.__get_answer__(question)

      metadata_raw = [doc['chapter_name'] for doc in context]
      metadata_cleaned = [ast.literal_eval(item) for item in metadata_raw]

      print('User:', question)
      print('System:', llm_response.choices[0].message.content)
      if "don't know" not in llm_response.choices[0].message.content:
        print('Resources:', [chapter for doc in metadata_cleaned for chapter in doc])

In [152]:
bot = QuestionAnsweringBot(chunked_data_corpus_df)

# Testing with different prompts

In [153]:
question = "What is the title of Chapter 64?"
bot.form_response(question)

User: What is the title of Chapter 64?
System: The title of Chapter 64 is "The Beggar".
Resources: ['Chapter 64. The Beggar', 'Chapter 1. Marseilles—The Arrival']


In [154]:
question = "Who is the current president of US?"
bot.form_response(question)

User: Who is the current president of US?
System: I don't know. The passage does not provide information about the current president of the US.


In [155]:
question = "Who is the author of the book The Count of Monte Cristo?"
bot.form_response(question)

User: Who is the author of the book The Count of Monte Cristo?
System: The author of the book "The Count of Monte Cristo" is Alexandre Dumas, in collaboration with Auguste Maquet.
Resources: ['Chapter 1. Marseilles—The Arrival', 'Chapter 41. The Presentation']


In [156]:
question = "Who is Monte Cristo?"
bot.form_response(question)

User: Who is Monte Cristo?
System: Monte Cristo is a mysterious and wealthy count who has recently moved to Paris.
Resources: ['Chapter 46. Unlimited Credit', 'Chapter 47. The Dappled Grays']


In [157]:
question = "Tell me about all the main identites in Monte Cristo?"
bot.form_response(question)

User: Tell me about all the main identites in Monte Cristo?
System: The main identities in Monte Cristo are:

1. Edmond Dantès: The protagonist of the story, a young and successful merchant sailor who is falsely accused of treason and imprisoned for years.
2. Monte Cristo (Count of): Edmond Dantès after his escape from prison, who uses his vast wealth and knowledge to exact revenge on those who wronged him.
3. Mercédès: Edmond Dantès' fiancée, who betrays him by marrying his rival, Fernand Mondego.
4. Fernand Mondego: Mercédès' new husband, who is jealous of Edmond Dantès and plots against him, leading to his imprisonment.
5. Danglars: A wealthy shipowner who is resentful of Edmond Dantès' success and is responsible for the initial betrayal that leads to his imprisonment.
6. Villefort: A prosecutor who is corrupt and has a hidden past, which becomes an important part of the story.
7. Morrel: A kind and honest shipowner who is a friend of Edmond Dantès and has a daughter named Julie.
8.

In [158]:
question = "Name me all the persons under whose name Monte Cristo appeared"
bot.form_response(question)

User: Name me all the persons under whose name Monte Cristo appeared
System: Based on the provided text, the following are the names under which Monte Cristo appeared:

1. The Abbé Busoni
2. Edmond Dantès
3. Abbé Villefort
4. Lord Wilmore
5. Pierre Morrel

Please let me know if you'd like me to extract any other information from the text.
Resources: ['Chapter 84. Beauchamp', 'Chapter 46. Unlimited Credit', 'Chapter 45. The Rain of Blood']


In [159]:
question = "How many years does Edmon Dantes spent in prison?"
bot.form_response(question)

User: How many years does Edmon Dantes spent in prison?
System: According to the context, Edmond Dantès was arrested on February 28, 1815, and since he was 19 years old then, he would be approximately 26-27 years old when he starts talking to the unknown prisoner (No. 27). This means that Dantès spent at least 14-15 years in prison, not knowing about the outside world and its changes, including Napoleon's abdication and exile to the Island of Elba.
Resources: ['Chapter 15. Number 34 and Number 27', 'Chapter 22. The Smugglers']


In [160]:
question = "What is the name of the ship Edmon Dantes used to work on while working in Morrels company?"
bot.form_response(question)

User: What is the name of the ship Edmon Dantes used to work on while working in Morrels company?
System: The name of the ship Edmond Dantès used to work on while working in Morrel's company is the Pharaon.
Resources: ['Chapter 5. The Marriage Feast', 'Chapter 1. Marseilles—The Arrival']


In [162]:
question = "What is the title of Chapter 93?"
bot.form_response(question)

User: What is the title of Chapter 93?
System: Chapter 93 of the book "The Count of Monte Cristo" is titled "Valentine".
Resources: ['Chapter 1. Marseilles—The Arrival', 'Chapter 33. Roman Bandits']


As we can see, all answers were correct.

# Testing different retrievers

### Only BM25

In [193]:
bot_bm25 = QuestionAnsweringBot(chunked_data_corpus_df, enable_bm25=True, enable_dense=False, enable_rerank=False, top_k_bm25=2)

In [197]:
print("--- QUESTION 1 ---")
question = "What is the title of Chapter 64?"
bot_bm25.form_response(question)

--- QUESTION 1 ---
User: What is the title of Chapter 64?
System: The title of Chapter 64 is "The Beggar".
Resources: ['Chapter 1. Marseilles—The Arrival', 'Chapter 64. The Beggar']


In [196]:
print("--- QUESTION 2 ---")
question = "Tell me about all the main identites in Monte Cristo?"
bot_bm25.form_response(question)

--- QUESTION 2 ---
User: Tell me about all the main identites in Monte Cristo?
System: What a tale of intrigue and romance! Monte Cristo is a masterpiece of literature, and its characters are just as fascinating as the plot. Here's a brief rundown of the main identities you'll find in the novel:

**Edmond Dantès (alias The Count of Monte Cristo)**: The protagonist of the novel, Edmond Dantès is a young and successful merchant sailor who becomes embroiled in a plot to destroy his life. After being wrongfully imprisoned for many years, Dantès manages to escape and seeks revenge on those who wronged him. He assumes the identity of the enigmatic and wealthy Count of Monte Cristo, using his newfound wealth and influence to orchestrate a series of dramatic events.

**Abélino Morrel (father of Mercédès)**: Morrel is Dantès' former captain and mentor, who saves Dantès' life on multiple occasions. He is a good and honest man, who is deeply devoted to his daughter Mercédès and her potential well

In [195]:
print("--- QUESTION 3 ---")
question = "How many years does Edmon Dantes spent in prison?"
bot_bm25.form_response(question)

--- QUESTION 3 ---
User: How many years does Edmon Dantes spent in prison?
System: The question mentions Edmond Dantes, not Edmon Dantes. According to the text, Edmond Dantes spent 14 years in prison.
Resources: ['Chapter 89. The Night', 'Chapter 90. The Meeting', 'Chapter 41. The Presentation', 'Chapter 40. The Breakfast']


In [194]:
print("--- QUESTION 4 ---")
question = "Why Edmond Dantes was in prison?"
bot_bm25.form_response(question)

--- QUESTION 4 ---
User: Why Edmond Dantes was in prison?
System: In this context, it is not specified why Edmond Dantes was in prison. According to what is told by the abbé, Edmond was a young sailor who was imprisoned and died in prison, but the reason for his imprisonment was not disclosed.
Resources: ['Chapter 4. Conspiracy', 'Chapter 3. The Catalans', 'Chapter 26. The Pont du Gard Inn']


### Only Dense

In [171]:
bot_dense = QuestionAnsweringBot(chunked_data_corpus_df, enable_bm25=False, enable_dense=True, enable_rerank=False, top_n_dense=3)

In [172]:
print("--- QUESTION 1 ---")
question = "What is the title of Chapter 64?"
bot_dense.form_response(question)

--- QUESTION 1 ---
User: What is the title of Chapter 64?
System: The title of Chapter 64 is "A Conjugal Scene".
Resources: ['Chapter 87. The Challenge', 'Chapter 86. The Trial', 'Chapter 1. Marseilles—The Arrival', 'Chapter 106. Dividing the Proceed', 'Chapter 105. The Cemetery of Père-Lachaise']


In [173]:
print("--- QUESTION 2 ---")
question = "Tell me about all the main identites in Monte Cristo?"
bot_dense.form_response(question)

--- QUESTION 2 ---
User: Tell me about all the main identites in Monte Cristo?
System: What an epic book!

In Alexandre Dumas' classic novel "The Count of Monte Cristo", there are many complex and interesting characters. Here's a brief summary of the main identities:

**The Main Protagonist:**

* Edmond Dantès: The young and enthusiastic merchant sailor who is wrongfully accused and imprisoned. He later becomes the wealthy and mysterious Count of Monte Cristo.

**The Main Antagonists:**

* Danglars: A wealthy and ambitious shipowner who is jealous of Dantès' success and becomes a main antagonist.
* Fernand Mondego: Dantès' supposed friend who is actually his rival in love and becomes a rival in business and social status.
* Villefort: A corrupt and ambitious prosecutor who sends Dantès to prison and is later manipulated by the Count of Monte Cristo.

**Supporting Characters:**

* Mercédès: Dantès' fiancée who becomes the object of desire for Fernand Mondego.
* Haydée: The daughter of A

In [175]:
print("--- QUESTION 3 ---")
question = "How many years does Edmon Dantes spent in prison?"
bot_dense.form_response(question)

--- QUESTION 3 ---
User: How many years does Edmon Dantes spent in prison?
System: Based on the provided context, it's not explicitly stated how many years Edmond Dantès spent in prison. However, in the novel "The Count of Monte Cristo" by Alexandre Dumas, Edmond Dantès is falsely accused of treason and imprisoned for 14 years (from 1807 to 1821) before his escape and the events that unfold.
Resources: ['Chapter 17. The Abbé’s Chamber', 'Chapter 18. The Treasure', 'Chapter 8. The Château d’If', 'Chapter 16. A Learned Italian']


In [190]:
print("--- QUESTION 4 ---")
question = "Why Edmond Dantes was in prison?"
bot_dense.form_response(question)

--- QUESTION 4 ---
User: Why Edmond Dantes was in prison?
System: Edmond Dantes was in prison for being falsely accused of treason against Napoleon Bonaparte, the then ruler of France. He was betrayed by his friend Fernand Mondego, who was jealous of Dantes' success and social rise. Dantes was imprisoned on the eve of his wedding, without a chance to defend himself, and was left to rot in the Châteauneuf prison.
Resources: ['Chapter 17. The Abbé’s Chamber', 'Chapter 18. The Treasure', 'Chapter 23. The Island of Monte Cristo', 'Chapter 24. The Secret Cave', 'Chapter 15. Number 34 and Number 27']


The question answering bot with BM25 retriever gave a better answer to QUESTION 1, but didn't answer QUESTION 4, unlike the bot with the dense retriever. Also bot with the dense retriever made mistake in aswer to QUESTION 1. In summary, BM25 retriever doesn't good if question has contextual aspects, that cannot be captured by an exact keyword match. 

# Testing reranker

### Without reranker

In [208]:
bot_no_rerank = QuestionAnsweringBot(chunked_data_corpus_df, enable_rerank=False, top_n_dense=2)

In [210]:
print("--- QUESTION 1 ---")
question = "What is the title of Chapter 64?"
bot_no_rerank.form_response(question)

--- QUESTION 1 ---
User: What is the title of Chapter 64?
System: A nice question!

According to the provided context, Chapter 64's title is: **"A Conjugal Scene"**.
Resources: ['Chapter 1. Marseilles—The Arrival', 'Chapter 114. Peppino']


In [215]:
print("--- QUESTION 2 ---")
question = "Tell me about all the main identites in Monte Cristo?"
bot_no_rerank.form_response(question)

--- QUESTION 2 ---
User: Tell me about all the main identites in Monte Cristo?
System: Monte Cristo is a novel by Alexandre Dumas, and the story is quite complex with multiple plots and characters. Here's a summary of the main identities in Monte Cristo:

* **Edmond Dantès**: The protagonist of the novel, a young and successful sailor who is falsely accused of treason and imprisoned. He seeks revenge and becomes the wealthy and powerful Monte Cristo.
* **Albert de Morcerf**: Edmond's friend and companion, who becomes embroiled in the plot and is eventually accused of theft and murder.
* **Fernand Mondego**: A rival of Edmond's who is responsible for his imprisonment and seeks to profit from his misfortune.
* **Danglars**: A wealthy banker who is in debt to Edmond and seeks to harm him out of jealousy and revenge.
* **Villefort**: A prosecutor who holds Edmond's fate in his hands and eventually becomes embroiled in the plot to avenge Edmond's imprisonment.
* **Mercédès Iguanada**: Edmon

In [217]:
print("--- QUESTION 3 ---")
question = "How many years does Edmon Dantes spent in prison?"
bot_no_rerank.form_response(question)

--- QUESTION 3 ---
User: How many years does Edmon Dantes spent in prison?
System: Edmon Dantes, the protagonist, spent 14 years in prison.
Resources: ['Chapter 16. A Learned Italian', 'Chapter 113. The Past', 'Chapter 114. Peppin']


In [216]:
print("--- QUESTION 4 ---")
question = "Why Edmond Dantes was in prison?"
bot_no_rerank.form_response(question)

--- QUESTION 4 ---
User: Why Edmond Dantes was in prison?
System: According to the context, Edmond Dantes was imprisoned because he was accused of being involved in a Bonapartist plot to assassinate Napoleon Bonaparte, which was a false accusation set up by his jealous colleague, Fernand Mondego, who was in love with Mercédès, Dantès' fiancée.
Resources: ['Chapter 17. The Abbé’s Chamber', 'Chapter 18. The Treasure', 'Chapter 23. The Island of Monte Cristo', 'Chapter 24. The Secret Cave']


### With reranker

In [218]:
bot_rerank = QuestionAnsweringBot(chunked_data_corpus_df)

In [219]:
print("--- QUESTION 1 ---")
question = "What is the title of Chapter 64?"
bot_rerank.form_response(question)

--- QUESTION 1 ---
User: What is the title of Chapter 64?
System: The title of Chapter 64 is "The Beggar."
Resources: ['Chapter 64. The Beggar', 'Chapter 1. Marseilles—The Arrival']


In [220]:
print("--- QUESTION 2 ---")
question = "Tell me about all the main identites in Monte Cristo?"
bot_rerank.form_response(question)

--- QUESTION 2 ---
User: Tell me about all the main identites in Monte Cristo?
System: Monte Cristo (Edmond Dantès) is a novel written by Alexandre Dumas. The main characters in the novel can be categorized as follows:

**Plot-Driving Characters:**

1. **Edmond Dantès** (also known as the Count of Monte Cristo): The protagonist of the novel, a young and successful merchant sailor who is betrayed by his friends and framed for treason.
2. **Abbé Faria**: A kind-hearted and wise abbé who befriends Dantès in prison and helps him to escape. He is also the one who reveals to Dantès that he will find a vast treasure in the grotto of the Château d'If.
3. **Morrel**: A wealthy shipowner who employs Dantès. He is a kind and honest man who becomes a friend and benefactor to Dantès.
4. **Danglars**: A rival shipowner who is jealous of Morrel's success and seeks to ruin Dantès. He is also a gambler and a philanderer.
5. **Villefort**: A corrupt and ambitious prosecutor who is responsible for Dantès

In [222]:
print("--- QUESTION 3 ---")
question = "How many years does Edmon Dantes spent in prison?"
bot_rerank.form_response(question)

--- QUESTION 3 ---
User: How many years does Edmon Dantes spent in prison?
System: Edmond Dantès has been a prisoner for fourteen years, since the 28th of February 1815.
Resources: ['Chapter 15. Number 34 and Number 27', 'Chapter 22. The Smugglers']


In [224]:
print("--- QUESTION 4 ---")
question = "Why Edmond Dantes was in prison?"
bot_rerank.form_response(question)

--- QUESTION 4 ---
User: Why Edmond Dantes was in prison?
System: Edmond Dantes was in prison because he was falsely accused of being involved in aBonapartist plot and was sentenced to life imprisonment on the island of If, also known as the Château d'If.
Resources: ['Chapter 17. The Abbé’s Chamber', 'Chapter 18. The Treasure', 'Chapter 15. Number 34 and Number 27']


Looking at the answers, we can conclude that the reranker improved the accuracy and content of the answers.